In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
import tkinter as tk
import imutils
from PIL import ImageTk, Image
import tkinter.filedialog as filedialog
from tkinter import messagebox
import random

# Load the pre-trained model for emotion detection
model = load_model('C:\\Users\\91914\\Downloads\\model.h5')

# Define the list of emotions
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Emotion-recommendation mapping
recommendations = {
    'Angry': [
        'Take deep breaths and count to 10 before responding.',
        'Engage in physical activities like exercise or sports to release pent-up anger.',
        'Express your anger through writing or artwork.',
    ],
    'Disgust': [
        'Focus on positive aspects and things you are grateful for.',
        'Practice mindfulness or meditation to shift your focus away from negative feelings.',
        'Engage in activities that bring you joy and happiness.',
    ],
    'Fear': [
        'Take small steps to face your fears and seek support if needed.',
        'Challenge negative thoughts and replace them with positive affirmations.',
        'Practice relaxation techniques like deep breathing or progressive muscle relaxation.',
    ],
    'Happy': [
        'Celebrate your happiness and spread positivity to others.',
        'Engage in activities that bring you joy and fulfillment.',
        'Express gratitude and appreciation for the good things in your life.',
    ],
    'Sad': [
        'Reach out to someone you trust and talk about how you feel.',
        'Engage in self-care activities like taking a warm bath or reading a book.',
        'Practice self-compassion and allow yourself to grieve and heal.',
    ],
    'Surprise': [
        'Embrace the unexpected and stay open to new experiences.',
        'Take it as an opportunity for growth and learning.',
        'Embrace the feeling of excitement and anticipation that comes with surprises.',
    ],
    'Neutral': [
        'Take some time for self-reflection and self-care.',
        'Engage in activities that bring you peace and relaxation.',
        'Explore new hobbies or interests to find inspiration and meaning.',
    ]
}

# Chatbot responses
chatbot_responses = [
    'Tell me more about it.',
    'How does that make you feel?',
    'What do you think you should do?',
    'I understand. It can be challenging.',
    'Is there anything else you would like to share?'
]

# Function to perform emotion detection on an image
def detect_emotion(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (48, 48))
    img = img.reshape((1, 48, 48, 1))
    img = img.astype('float32') / 255

    prediction = model.predict(img)
    max_index = np.argmax(prediction[0])
    emotion = emotions[max_index]
    accuracy = round(prediction[0][max_index] * 100, 2)
    return emotion, accuracy

# Function to handle image upload
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        img = cv2.imread(file_path)
        emotion, accuracy = detect_emotion(img)
        result_label.config(text=f'Emotion: {emotion}, Accuracy: {accuracy}%')

        # Display the uploaded image in the GUI
        display_uploaded_image(img)

        # Display recommendations for the detected emotion
        display_recommendations(emotion)

# Function to start live emotion detection
def start_live_detection():
    global live_detection_active
    live_detection_active = True

    video_capture = cv2.VideoCapture(0)

    def update_frame():
        ret, frame = video_capture.read()
        frame = cv2.flip(frame, 1)
        emotion, accuracy = detect_emotion(frame)
        result_label.config(text=f'Emotion: {emotion}, Accuracy: {accuracy}%')

        # Display the frame in the GUI
        display_live_image(frame)

        if live_detection_active:
            video_label.after(10, update_frame)
        else:
            video_capture.release()

    update_frame()

# Function to stop live emotion detection
def stop_live_detection():
    global live_detection_active
    live_detection_active = False

# Function to display the recommendations for a specific emotion
def display_recommendations(emotion):
    recommendations_text.delete('1.0', tk.END)
    if emotion in recommendations:
        recommendations_text.insert(tk.END, recommendations[emotion])
    else:
        recommendations_text.insert(tk.END, 'No recommendations available.')

# Function to display the uploaded image
def display_uploaded_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img = img.resize((250, 250), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    uploaded_image_label.config(image=img)
    uploaded_image_label.image = img

# Function to display the live detection frame
def display_live_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (250, 250))
    img = Image.fromarray(img)
    img = ImageTk.PhotoImage(img)
    video_label.config(image=img)
    video_label.image = img

# Function to ask a question to the chatbot
def ask_question():
    question = chatbot_entry.get().strip()
    if question:
        chatbot_entry.delete(0, tk.END)
        response = random.choice(chatbot_responses)
        messagebox.showinfo('Chatbot', f'You asked: {question}\n\nChatbot says: {response}')
        chatbot_response_label.config(text=response)

# Create the GUI
root = tk.Tk()
root.title("Emotion Detection")
root.geometry("900x600")

# Add background image
background_image = ImageTk.PhotoImage(file="C:\\Users\\91914\\Downloads\\cmaq83wr.png")
background_label = tk.Label(root, image=background_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Upload Image Section
upload_frame = tk.Frame(root)
upload_frame.pack(pady=10)

upload_label = tk.Label(upload_frame, text='Upload Image:')
upload_label.pack(side=tk.LEFT)

upload_button = tk.Button(upload_frame, text='Browse', command=upload_image)
upload_button.pack(side=tk.LEFT, padx=10)

# Live Detection Section
live_detection_frame = tk.Frame(root)
live_detection_frame.pack(pady=10)

live_detection_label = tk.Label(live_detection_frame, text='Live Detection:')
live_detection_label.pack(side=tk.LEFT)

start_button = tk.Button(live_detection_frame, text='Start', command=start_live_detection)
start_button.pack(side=tk.LEFT, padx=10)

stop_button = tk.Button(live_detection_frame, text='Stop', command=stop_live_detection)
stop_button.pack(side=tk.LEFT)

# Result Section
result_frame = tk.Frame(root)
result_frame.pack(pady=10)

result_label = tk.Label(result_frame, text='', font=('Helvetica', 14, 'bold'))
result_label.pack()

# Recommendations Section
recommendations_frame = tk.Frame(root)
recommendations_frame.pack(pady=10)

recommendations_label = tk.Label(recommendations_frame, text='Recommendation:', font=('Helvetica', 12, 'bold'))
recommendations_label.pack(side=tk.LEFT)

recommendations_text = tk.Text(recommendations_frame, font=('Helvetica', 12), width=50, height=6)
recommendations_text.pack(side=tk.LEFT, padx=10)

scrollbar = tk.Scrollbar(recommendations_frame, orient=tk.VERTICAL, command=recommendations_text.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
recommendations_text.configure(yscrollcommand=scrollbar.set)

# Chatbot Section
chatbot_frame = tk.Frame(root)
chatbot_frame.pack(pady=10)

chatbot_title_label = tk.Label(chatbot_frame, text="I'm here to talk!", font=('Helvetica', 16, 'bold'))
chatbot_title_label.pack()

chatbot_entry = tk.Entry(chatbot_frame, font=('Helvetica', 12))
chatbot_entry.pack(pady=10)

ask_button = tk.Button(chatbot_frame, text='Ask', command=ask_question)
ask_button.pack()

chatbot_response_label = tk.Label(chatbot_frame, text='', font=('Helvetica', 12))
chatbot_response_label.pack(pady=10)

# Image Display Section
image_display_frame = tk.Frame(root)
image_display_frame.pack()

uploaded_image_label = tk.Label(image_display_frame)
uploaded_image_label.pack(side=tk.LEFT, padx=10)

video_label = tk.Label(image_display_frame)
video_label.pack(side=tk.LEFT, padx=10)

root.mainloop()
